In [ ]:
# Standard Library
import ast
import json
import random
import re
import time
from datetime import timedelta

# Third-Party Libraries
import folium
from folium.plugins import HeatMap
import geopandas as gpd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from tqdm import tqdm

# Geospatial Libraries
from shapely.geometry import Point, LineString, mapping
from shapely.ops import transform
from pyproj import Transformer

# Machine Learning / Clustering
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from scipy.spatial import ConvexHull




In [ ]:
#loading dataset
df = pd.read_csv('traffic_df.csv')


In [ ]:
# Preprocessing. Creating columns saving each observation's previous lat and lon. Also removing obs near ports
# Extract consecutive latitude/longitude pairs
df['prev_LAT'] = df['LATITUDE'].shift(1)
df['prev_LON'] = df['LONGITUDE'].shift(1)
#removing datapoints near ports
df_filtered = df[df['PORT'] == 'no_port']

In [ ]:
def compute_cog(row): #compute course over ground using a bearing formula
    if pd.isna(row['prev_LAT']):
        return np.nan
    lat1, lon1 = row['prev_LAT'], row['prev_LON']
    lat2, lon2 = row['LATITUDE'], row['LONGITUDE']
    
    # Bearing calculation 
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    delta_lon = lon2_rad - lon1_rad
    x = np.sin(delta_lon) * np.cos(lat2_rad)
    y = np.cos(lat1_rad) * np.sin(lat2_rad) - np.sin(lat1_rad) * np.cos(lat2_rad) * np.cos(delta_lon)
    cog_rad = np.arctan2(x, y)
    cog_deg = (np.degrees(cog_rad) + 360) % 360
    return cog_deg



#calling the function and applying it to calculate COG manually
df_filtered['computed_COG'] = df_filtered.apply(compute_cog, axis=1)

In [ ]:
#Function to calculate consecutive rows (grouped by mmsi and sorted by timestamp) changes of course

def find_large_change_of_COR(df):
    df['DATE TIME (UTC)'] = pd.to_datetime(df['DATE TIME (UTC)'])
    df = df.sort_values(by=['MMSI', 'DATE TIME (UTC)'])
    
    # calculate difference in absolute values and save results in a new column
    df['CourseDiff'] = df.groupby('MMSI')['computed_COG'].diff().abs()
    large_course_change = df
    
    return large_course_change

In [ ]:
#Calling the function and filtering to identified course changes larger than 90 degrees.
large_changes_course_anomalies = find_large_change_of_COR(df_filtered) 
large_changes_course_anomalies = large_changes_course_anomalies[large_changes_course_anomalies['CourseDiff'] >= 90]

In [ ]:
# Load infrastructure data from files
def load_dict_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    content = re.sub(r'"([^"]+)"\s*:', r'"\1":', content)
    return ast.literal_eval("{" + content + "}")

# Load cable and pipeline data
telco_cables_dict = load_dict_from_file('telco_cables.txt')
pow_cables_dict = load_dict_from_file('power_cables.txt')
gas_pipe_dict = load_dict_from_file('gas_pipe.txt')

# Add new columns initialized to 0
large_changes_course_anomalies['cc_near_telco_cable'] = 0
large_changes_course_anomalies['cc_near_power_cable'] = 0
large_changes_course_anomalies['cc_near_gas_pipe'] = 0

# Set up coordinate transformers for metric and geographic systems
project_to_meters = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True).transform
project_to_degrees = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True).transform

# Create map centered on first telco cable point
first_cable_points = list(telco_cables_dict.values())[0]
center_point = first_cable_points[0]
m = folium.Map(location=[center_point[1], center_point[0]], zoom_start=6)

# Add telco cables and 10km buffers to map
for cable_name, cable_points in telco_cables_dict.items():
    cable_line = LineString(cable_points)
    cable_line_proj = transform(project_to_meters, cable_line)
    buffer_proj = cable_line_proj.buffer(10000)
    buffer_latlon = transform(project_to_degrees, buffer_proj)
    folium.PolyLine(
        locations=[(pt[1], pt[0]) for pt in cable_points],
        color='red', weight=3, opacity=0.8, tooltip=cable_name
    ).add_to(m)
    folium.GeoJson(
        mapping(buffer_latlon),
        style_function=lambda x: {
            'fillColor': 'blue', 'color': 'blue', 'weight': 1, 'fillOpacity': 0.3
        }
    ).add_to(m)

# Add power cables and 10km buffers to map
for cable_name, cable_points in pow_cables_dict.items():
    cable_line = LineString(cable_points)
    cable_line_proj = transform(project_to_meters, cable_line)
    buffer_proj = cable_line_proj.buffer(10000)
    buffer_latlon = transform(project_to_degrees, buffer_proj)
    folium.PolyLine(
        locations=[(pt[1], pt[0]) for pt in cable_points],
        color='orange', weight=3, opacity=0.8, tooltip=cable_name
    ).add_to(m)
    folium.GeoJson(
        mapping(buffer_latlon),
        style_function=lambda x: {
            'fillColor': 'yellow', 'color': 'yellow', 'weight': 1, 'fillOpacity': 0.3
        }
    ).add_to(m)

# Add gas pipelines and 10km buffers to map
for cable_name, cable_points in gas_pipe_dict.items():
    cable_line = LineString(cable_points)
    cable_line_proj = transform(project_to_meters, cable_line)
    buffer_proj = cable_line_proj.buffer(10000)
    buffer_latlon = transform(project_to_degrees, buffer_proj)
    folium.PolyLine(
        locations=[(pt[1], pt[0]) for pt in cable_points],
        color='black', weight=3, opacity=0.8, tooltip=cable_name
    ).add_to(m)
    folium.GeoJson(
        mapping(buffer_latlon),
        style_function=lambda x: {
            'fillColor': 'green', 'color': 'green', 'weight': 1, 'fillOpacity': 0.3
        }
    ).add_to(m)

# Check each anomaly point if it is within any cable buffer
for _, row in large_changes_course_anomalies.dropna(subset=['LATITUDE', 'LONGITUDE']).iterrows():
    lon, lat = row['LONGITUDE'], row['LATITUDE']
    point = Point(lon, lat)
    point_added = False

    # Check telco cables
    for cable_name, cable_points in telco_cables_dict.items():
        cable_line = LineString(cable_points)
        cable_buffer = cable_line.buffer(0.1)
        if cable_buffer.contains(point):
            folium.CircleMarker(
                location=(lat, lon), radius=1, color='purple',
                fill=True, fill_color='purple', fill_opacity=0.8
            ).add_to(m)
            point_added = True
            large_changes_course_anomalies.at[_, 'cc_near_telco_cable'] = 1

    # Check gas pipelines
    for cable_name2, cable_points2 in gas_pipe_dict.items():
        cable_line2 = LineString(cable_points2)
        cable_buffer2 = cable_line2.buffer(0.1)
        if cable_buffer2.contains(point):
            folium.CircleMarker(
                location=(lat, lon), radius=1, color='blue',
                fill=True, fill_color='blue', fill_opacity=0.8
            ).add_to(m)
            point_added = True
            large_changes_course_anomalies.at[_, 'cc_near_gas_pipe'] = 1

    # Check power cables
    for cable_name3, cable_points3 in pow_cables_dict.items():
        cable_line3 = LineString(cable_points3)
        cable_buffer3 = cable_line3.buffer(0.1)
        if cable_buffer3.contains(point):
            folium.CircleMarker(
                location=(lat, lon), radius=1, color='orange',
                fill=True, fill_color='orange', fill_opacity=0.8
            ).add_to(m)
            point_added = True
            large_changes_course_anomalies.at[_, 'cc_near_power_cable'] = 1

    # If not near any cable, keep columns at 0
    if not point_added:
        large_changes_course_anomalies.at[_, 'cc_near_telco_cable'] = 0
        large_changes_course_anomalies.at[_, 'cc_near_gas_pipe'] = 0
        large_changes_course_anomalies.at[_, 'cc_near_power_cable'] = 0

# Save map to HTML file
m.save("all_cables_large_changes_anomalies_map.html")


In [ ]:
# save large changes course anomalies to CSV
large_changes_course_anomalies.to_csv("large_changes_course_anomalies.csv", index=False)

In [ ]:

# Sort by MMSI and timestamp
filter_large_changes = large_changes_course_anomalies.sort_values(['MMSI', 'DATE TIME (UTC)'])

# Parameters
window_size = timedelta(hours=12)
step_size = timedelta(hours=6)

# Container for results
sliding_window_results = []

# Group by MMSI
for mmsi, group in tqdm(filter_large_changes.groupby('MMSI')):
    group = group.reset_index(drop=True)
    
    start_time = group['DATE TIME (UTC)'].min()
    end_time = group['DATE TIME (UTC)'].max()
    
    current_start = start_time
    
    while current_start + window_size <= end_time:
        current_end = current_start + window_size
        
        window_data = group[
            (group['DATE TIME (UTC)'] >= current_start) &
            (group['DATE TIME (UTC)'] < current_end)
        ]
        
        if not window_data.empty:
            sliding_window_results.append({
                'MMSI': mmsi,
                'window_start': current_start,
                'window_end': current_end,
                'observation_count': len(window_data),
                'start_latitude': window_data.iloc[0]['LATITUDE'],
                'start_longitude': window_data.iloc[0]['LONGITUDE'],
                'end_latitude': window_data.iloc[-1]['LATITUDE'],
                'end_longitude': window_data.iloc[-1]['LONGITUDE']
            })
        
        current_start += step_size

# Final DataFrame
sliding_result_df = pd.DataFrame(sliding_window_results)


# Display the result
display(sliding_result_df)

In [ ]:
# Filter results to keep only those with at least 6 observations namely course changes larger than 90 degrees
result = sliding_result_df[sliding_result_df['observation_count'] >= 6]

In [ ]:
# Calcola la media di start_latitude e end_latitude
result['mean_latitude'] = (result['start_latitude'] + result['end_latitude']) / 2

# Calcola la media di start_longitude e end_longitude
result['mean_longitude'] = (result['start_longitude'] + result['end_longitude']) / 2



In [ ]:
# Add new columns, set to 0
result['zz_near_telco_cable'] = 0
result['zz_near_power_cable'] = 0
result['zz_near_gas_pipe'] = 0

# Set up transformers for metric and geographic coordinates
project_to_meters = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True).transform
project_to_degrees = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True).transform

# Create map centered on first cable point
first_cable_points = list(telco_cables_dict.values())[0]
center_point = first_cable_points[0]
m = folium.Map(location=[center_point[1], center_point[0]], zoom_start=6)

# Add telco cables and buffers to map
for cable_name, cable_points in telco_cables_dict.items():
    cable_line = LineString(cable_points)
    cable_line_proj = transform(project_to_meters, cable_line)
    buffer_proj = cable_line_proj.buffer(10000)
    buffer_latlon = transform(project_to_degrees, buffer_proj)
    folium.PolyLine(
        locations=[(pt[1], pt[0]) for pt in cable_points],
        color='red', weight=3, opacity=0.8, tooltip=cable_name
    ).add_to(m)
    folium.GeoJson(
        mapping(buffer_latlon),
        style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 1, 'fillOpacity': 0.3}
    ).add_to(m)

# Add power cables and buffers to map
for cable_name, cable_points in pow_cables_dict.items():
    cable_line = LineString(cable_points)
    cable_line_proj = transform(project_to_meters, cable_line)
    buffer_proj = cable_line_proj.buffer(10000)
    buffer_latlon = transform(project_to_degrees, buffer_proj)
    folium.PolyLine(
        locations=[(pt[1], pt[0]) for pt in cable_points],
        color='orange', weight=3, opacity=0.8, tooltip=cable_name
    ).add_to(m)
    folium.GeoJson(
        mapping(buffer_latlon),
        style_function=lambda x: {'fillColor': 'yellow', 'color': 'yellow', 'weight': 1, 'fillOpacity': 0.3}
    ).add_to(m)

# Add gas pipes and buffers to map
for cable_name, cable_points in gas_pipe_dict.items():
    cable_line = LineString(cable_points)
    cable_line_proj = transform(project_to_meters, cable_line)
    buffer_proj = cable_line_proj.buffer(10000)
    buffer_latlon = transform(project_to_degrees, buffer_proj)
    folium.PolyLine(
        locations=[(pt[1], pt[0]) for pt in cable_points],
        color='black', weight=3, opacity=0.8, tooltip=cable_name
    ).add_to(m)
    folium.GeoJson(
        mapping(buffer_latlon),
        style_function=lambda x: {'fillColor': 'green', 'color': 'green', 'weight': 1, 'fillOpacity': 0.3}
    ).add_to(m)

# Check each point, mark if near a cable, update columns
for _, row in tqdm(result.dropna(subset=['mean_latitude', 'mean_longitude']).iterrows()):
    lon, lat = row['mean_longitude'], row['mean_latitude']
    point = Point(lon, lat)
    point_added = False
    for cable_name, cable_points in telco_cables_dict.items():
        cable_line = LineString(cable_points)
        cable_buffer = cable_line.buffer(0.1)
        if cable_buffer.contains(point):
            folium.CircleMarker(
                location=(lat, lon), radius=1, color='purple',
                fill=True, fill_color='purple', fill_opacity=0.8
            ).add_to(m)
            point_added = True
            result.at[_, 'zz_near_telco_cable'] = 1
            break
    for cable_name2, cable_points2 in gas_pipe_dict.items():
        cable_line2 = LineString(cable_points2)
        cable_buffer2 = cable_line2.buffer(0.1)
        if cable_buffer2.contains(point):
            folium.CircleMarker(
                location=(lat, lon), radius=1, color='blue',
                fill=True, fill_color='blue', fill_opacity=0.8
            ).add_to(m)
            point_added = True
            result.at[_, 'zz_near_gas_pipe'] = 1
            break
    for cable_name3, cable_points3 in pow_cables_dict.items():
        cable_line3 = LineString(cable_points3)
        cable_buffer3 = cable_line3.buffer(0.1)
        if cable_buffer3.contains(point):
            folium.CircleMarker(
                location=(lat, lon), radius=1, color='orange',
                fill=True, fill_color='orange', fill_opacity=0.8
            ).add_to(m)
            point_added = True
            result.at[_, 'zz_near_power_cable'] = 1
            break
    if not point_added:
        result.at[_, 'zz_near_telco_cable'] = 0
        result.at[_, 'zz_near_gas_pipe'] = 0
        result.at[_, 'zz_near_power_cable'] = 0

# Save map to HTML
m.save("all_cables_zigzag_map.html")

In [ ]:
# Save result to CSV
result.to_csv("zigzag_anomalies.csv", index=False)